**Table of contents**<a id='toc0_'></a>    
- [Building A Basic Chatbot](#toc1_)    
  - [Message History](#toc1_1_)    
  - [Prompt templates](#toc1_2_)    
  - [Managing the Conversation History](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Building A Basic Chatbot](#toc0_)

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.
Note that this chatbot that we build will only use the language model to have a conversation.


- Environment Setup: Loads environment variables and retrieves Groq API key.
    * **Environment Variables**: A method to securely store and access sensitive information like API keys using a `.env` file and the `python-dotenv` package.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() # Loading all the environment variables

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

- Model Initialization: Creates a ChatGroq instance using Gemma-2-9b model with the specified API key.
    - ChatGroq: A LangChain integration class that provides access to Groq's language models through their API, specifically configured here to use the Gemma-2-9b-It model for text generation tasks.

In [3]:
# Import ChatGroq for accessing Groq's language models
from langchain_groq import ChatGroq

# Initialize ChatGroq with Gemma model and API key
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002B4B92EBBF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002B4B918F740>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

- Direct Model Interaction: Sends a message to the Groq model to test basic communication.
- HumanMessage: A LangChain message type that formats user input for language model interactions.

In [11]:
# Test model with a simple human message
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi , My name is Duygu and I am a Chief AI Engineer")])

AIMessage(content="Hello Duygu, it's a pleasure to meet you!\n\nAs a Chief AI Engineer, I imagine you have a lot of exciting projects and challenges on your plate.  \n\nWhat kind of work are you currently focused on? Perhaps I can be of assistance with some AI-related tasks or offer insights based on my knowledge.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 23, 'total_tokens': 93, 'completion_time': 0.127272727, 'prompt_time': 0.00014234, 'queue_time': 0.014502727, 'total_time': 0.127415067}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ec014a83-5cf1-43b5-8ea8-d7bf38fa1eea-0', usage_metadata={'input_tokens': 23, 'output_tokens': 70, 'total_tokens': 93})

- Message Chain Testing: Tests model's ability to maintain context through a series of messages.
* **Conversation Context**: Demonstrates how to create a conversation chain using both HumanMessage and AIMessage objects to test the model's memory and context understanding.

In [ ]:
# Test model's context memory with a conversation sequence

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Duygu and I am a Chief AI Engineer"),
        AIMessage(content="Hello Duygu! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Duygu, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 99, 'total_tokens': 136, 'completion_time': 0.067272727, 'prompt_time': 0.003552545, 'queue_time': 0.011057773, 'total_time': 0.070825272}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f0a3d46-920a-40a3-8124-94083cf7d4c7-0', usage_metadata={'input_tokens': 99, 'output_tokens': 37, 'total_tokens': 136})

## <a id='toc1_1_'></a>[Message History](#toc0_)
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
%pip install langchain_community

- Chat History Implementation: Sets up a system to maintain conversation history across different sessions.
- Chat History System: A mechanism that stores and manages conversation history using LangChain's message history components, enabling the model to maintain context across multiple interactions within the same session.

In [13]:
# Set up chat history management system
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create storage for different chat sessions
store = {}

# Function to get or create chat history for a session
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Add message history capability to the model
with_message_history = RunnableWithMessageHistory(model, get_session_history)

* **Chat Session Management**: Explains how to handle conversation history for different chat sessions.

**Main Components**:
1. **get_session_history Function**:
   - Takes a session_id as input
   - Returns a chat history object for that session
   - Creates new history if session doesn't exist
   - Type hint `-> BaseChatMessageHistory` indicates return type

2. **Storage System**:
   - `store`: Dictionary that keeps chat histories
   - Key: session_id
   - Value: ChatMessageHistory object

3. **History Integration**:
   - `RunnableWithMessageHistory`: Combines model with chat history
   - Enables context-aware responses
   - Maintains separate history for each session

This system allows the model to remember past conversations within each unique session, making responses more contextually relevant.

In [14]:
# Set up config with session ID for message history tracking
config={"configurable":{"session_id":"chat1"}}

- Configuration Setup: Defines session configuration for chat history tracking.
- Session Configuration: A configuration object that specifies the session ID ("chat1") for tracking conversation history, allowing the system to maintain context for this specific chat session.

In [15]:
# Send message to model with history tracking enabled
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Duygu and I am a Chief AI Engineer")],
    config=config
)

- Message Invocation: Sends a message to the model while tracking conversation history.
- Historical Context: Invokes the model with message history enabled, using the specified session configuration to maintain conversation context.

In [16]:
response.content

"Hi Duygu,\n\nIt's nice to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.  \n\n"

- Context Testing: Tests the model's ability to recall information from previous messages in the conversation.
- Memory Verification: Asks the model to demonstrate its memory of earlier conversation by recalling user's name from chat history.

In [17]:
# Test model's memory by asking about previously mentioned information
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Duygu.  😊 \n\nI remember that from our introduction!  \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 96, 'total_tokens': 128, 'completion_time': 0.058181818, 'prompt_time': 0.004082993, 'queue_time': 0.010591244, 'total_time': 0.062264811}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-773c205e-8a7f-43b7-9fd8-20c7daa410d7-0', usage_metadata={'input_tokens': 96, 'output_tokens': 32, 'total_tokens': 128})

- Session Switch Test: Creates a new session to test memory isolation between different conversations.
- Context Separation: Demonstrates how different session IDs maintain separate conversation histories.

In [ ]:
# Test memory isolation with a new session ID
config2={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)

response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [19]:
# Initialize new session with different user information
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config2
)

response.content

"Hi John, it's nice to meet you!  👋  \n\nWhat can I do for you today?\n"

In [ ]:
# Verify memory retention in chat2 session
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config2
)

response.content

"Your name is John, remember?  😊  \n\nIs there anything else you'd like to talk about?\n"

## <a id='toc1_2_'></a>[Prompt templates](#toc0_)
- Prompt Templates help to turn raw user information into a format that the LLM can work with. 
- In this case, the raw user input is just a message, which we are passing to the LLM. 
    - First, add in a system message with some custom instructions (but still taking messages as input). 
    - Next, we'll add in more input besides just the messages.

- Prompt Template Creation: Configures a flexible ChatPromptTemplate with a system message and dynamic message placeholder.
- Chain Composition: Combines the prompt template with a language model to create a conversational processing chain.

In [21]:
# Import necessary classes for creating prompt templates
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Create a prompt template with system instruction and message history support
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Amnswer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

# Compose the chain by combining the prompt template with the model
chain = prompt | model

In [23]:
# Invoke the chain with an initial human message

chain.invoke({
    "messages": [HumanMessage(content="Hi My name is Duygu")]
})

AIMessage(content="Hi Duygu! It's nice to meet you. \n\nI'm glad you're here. What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 32, 'total_tokens': 69, 'completion_time': 0.067272727, 'prompt_time': 0.000307939, 'queue_time': 0.015237101, 'total_time': 0.067580666}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2124d0e-b5a3-4377-8112-f9b88e1c914f-0', usage_metadata={'input_tokens': 32, 'output_tokens': 37, 'total_tokens': 69})

* **Message History Configuration**: Sets up a runnable component that enables persistent message history tracking across conversation sessions.
* **Session History Management**: Integrates a mechanism to retrieve and store conversation history using the `get_session_history` function.

In [24]:
# Create a RunnableWithMessageHistory wrapper 
# This allows maintaining conversation context across multiple interactions

with_message_history = RunnableWithMessageHistory(
    chain,                  # Base conversational chain
    get_session_history     # Function to retrieve session-specific message history
)

- Session Configuration: Creates a configuration for a specific chat session with a unique session ID.
- Message Invocation: Sends a message to the chain with session-specific configuration.

In [ ]:
# Configure a unique session identifier for the conversation
config = {"configurable": {"session_id": "chat3"}}

# Invoke the chain with message history, using the specified session configuration
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Duygu")],  # Initial message from the user
    config=config  # Session-specific configuration
)

# Display the response from the model
response

AIMessage(content="Hi Duygu! 👋 \n\nIt's nice to meet you.  What can I do for you? \n\nI'm ready to answer your questions and help in any way I can. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 32, 'total_tokens': 79, 'completion_time': 0.085454545, 'prompt_time': 0.000304458, 'queue_time': 0.013443822, 'total_time': 0.085759003}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-898ad635-9de9-4db0-80f2-c004650227cb-0', usage_metadata={'input_tokens': 32, 'output_tokens': 47, 'total_tokens': 79})

In [26]:
# Session Context Verification:

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Duygu! 😊 I remember that you told me. \n\n\n\n'

In [27]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [ ]:
# Session Context Verification

response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Duygu")],
                       "language":"Turkish"})

response.content

'Merhaba Duygu!  Tanıştığıma memnun oldum.  Nasıl yardımcı olabilirim? 😊 \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {"configurable": {"session_id": "chat4"}}

repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Duygu")],
     "language":"French"},
    config=config
)

repsonse.content

"Bonjour Duygu, enchanté de te rencontrer ! Comment puis-je t'aider ? 😊  \n"

In [31]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], 
     "language": "French"},
    config=config,
)

In [ ]:
response.content

'Ton nom est Duygu. 😊  \n'

## <a id='toc1_3_'></a>[Managing the Conversation History](#toc0_)
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

- Message Trimming Configuration: Creates a message trimmer to manage conversation history and control token length.
- History Management Strategy: Configures a mechanism to limit and control the conversation context based on specific parameters.

In [33]:
# Import necessary classes for message manipulation
from langchain_core.messages import SystemMessage, trim_messages

# Create a message trimmer with specific configuration parameters
trimmer = trim_messages(
    max_tokens=45,            # Maximum number of tokens allowed in the context
    strategy="last",          # Keep the most recent messages
    token_counter=model,      # Use the model's token counting method
    include_system=True,      # Include system messages in the trimming process
    allow_partial=False,      # Disallow partial message inclusion
    start_on="human"          # Begin trimming from the last human message
)

This code demonstrates building a flexible conversational AI pipeline using LangChain, enabling dynamic message history management and context retention across different chat sessions with intelligent message trimming and session-based memory.

In [34]:
# Create a list of messages representing a conversation history
messages = [
    SystemMessage(content="you're a good assistant"),  # System instruction
    HumanMessage(content="hi! I'm bob"),               # First human message
    AIMessage(content="hi!"),                          # AI response
    HumanMessage(content="I like vanilla ice cream"),  # Another human message
    AIMessage(content="nice"),                         # AI response
    HumanMessage(content="whats 2 + 2"),               # Math question
    AIMessage(content="4"),                            # Math answer
    HumanMessage(content="thanks"),                    # Gratitude
    AIMessage(content="no problem!"),                  # AI acknowledgment
    HumanMessage(content="having fun?"),               # Casual question
    AIMessage(content="yes!"),                         # AI response
]

# Apply the trimmer to the message list
# This will reduce the messages based on the previously defined trimming strategy
trimmer.invoke(messages)

c:\Users\duygu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\duygu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\duygu\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In or

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

- Chain Composition: Constructs a complex runnable pipeline that dynamically trims messages before processing.
- Message Processing: Combines message history trimming, prompt generation, and model inference in a single workflow.

In [ ]:
# Import necessary utilities for function composition
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Create a sophisticated chain that:
# 1. Passes through existing messages
# 2. Trims messages using the previously defined trimmer
# 3. Applies the prompt template
# 4. Generates model response
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | prompt | model
)


# Invoke the chain with:
# - Existing message history
# - A new human message 
# - Additional context (language)
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like")],
        "language": "English"
    }
)


# Retrieve and display the model's response content
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?  🍦\n"

In [ ]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

# chat4 ten itibaren gecmis konusmalari hatirlar

"You asked what 2 + 2 equals. 😊  \n\n\n\nLet me know if you'd like to try another one!\n"

In [37]:
# Invoke the chain with a query about a previous conversation detail

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],  # Add new query to existing messages
        "language": "English",  # Specify language context
    }
)

# Display the model's response content
response.content

- Session Isolation Verification: Tests memory reset by starting a new chat session with a different session ID.
- Context Loss Demonstration: Verifies that the new session does not retain memory from previous conversations.

In [ ]:
# Configure a new session with a unique identifier
config = {"configurable": {"session_id": "chat5"}}  # New chat session

# Invoke the message history chain with the new session configuration
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],  # Query about name
        "language": "English",  # Language context
    },
    config=config,  # Use the new session configuration
)

# Retrieve and display the model's response
response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name. \n\nIs there anything else I can help you with?\n"

In [ ]:
# Invoke the message history chain in the new chat5 session
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],  # Query about past interaction
        "language": "English",  # Language context
    },
    config=config  # Use the new chat5 session configuration
)

# Retrieve and display the model's response
response.content

"As a helpful assistant, I have no memory of past conversations. Could you please tell me the math problem you'd like me to help with? 😄  \n\n"

In [ ]:
# END